In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json

In [ ]:
cid = ''
secret = ''
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=secret))

In [ ]:
# Example playlist: "wow im at the beach"
playlist_uri = 'spotify:playlist:5CF6KvWn85N6DoWufOjP5T'

In [ ]:
offset = 0
playlist_res = []

while True:
    response = sp.playlist_items(playlist_uri,
                                 offset=offset,
                                 fields='items.track.id,items.track.artists,items.track.name,items.track.album,total',
                                 additional_types=['track'])

    if len(response['items']) == 0:
        # Combine inner lists and exit loop
        # Todo: ask how this comprehension actually works
        playlist_res = [j for i in playlist_res for j in i]
        break

    playlist_res.append(response['items'])
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

In [ ]:
playlist_res

In [ ]:
# Create empty df to add json response to
df = pd.DataFrame(columns=['artist', 'track_name',
                  'id', 'album'], index=range(len(playlist_res)))

In [ ]:
# Add artist, album, song title, and ID to df
for i in range(len(playlist_res)):
    df['artist'].iloc[i] = playlist_res[i]['track']['artists'][0]['name']
    df['track_name'].iloc[i] = playlist_res[i]['track']['name']
    df['id'].iloc[i] = playlist_res[i]['track']['id']
    df['album'].iloc[i] = playlist_res[i]['track']['album']['name']


In [ ]:
df

In [ ]:
# Get audio features while avoiding rate limit

features = []
offset_min = 0
offset_max = 50

while True:
    if offset_min > len(df):
        break
    features.append(sp.audio_features(df['id'].iloc[offset_min:offset_max]))
    offset_min += 50
    offset_max += 50

In [ ]:
# Combine lists within previously created list and add to new DataFrame
features = [j for i in features for j in i]
df_features = pd.DataFrame(features)

In [ ]:
# Combine DataFrames
merged = df.combine_first(df_features.drop(columns=['track_href', 'analysis_url', 'uri', 'type', 'acousticness', 'danceability',
                                                    'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence']))

# Round tempos to nearest whole number for easier. Playlist generation works with tempo ranges, so decimal precision is unnecessary.
merged['tempo'] = round(merged['tempo']).astype(int)

In [ ]:
# Convert pitch class and mode integer columns to diatonic key signature column.
def key_to_camelot(df):
    df['key'] = df['key'].astype(str).replace({'-1': 'no key detected', '0': 'C', '1': 'D-flat', '2': 'D', '3': 'E-flat', '4': 'E',
                                                   '5': 'F', '6': 'F-sharp', '7': 'G', '8': 'A-flat', '9': 'A', '10': 'B-flat', '11': 'B'})

    df['mode'] = np.where(df['mode'] == 1, 'major', 'minor')
    df['key_signature'] = df['key'] + ' ' + df['mode']
    # Dictionary for mapping key signature integer value to pitch scale. Consult https://en.wikipedia.org/wiki/Pitch_class if you would like to confirm the translation.

    key_to_wheel = {'A-flat minor': '1A', 'B major': '1B', 'E-flat minor': '2A', 'F-sharp major': '2B', 'B-flat minor': '3A', 'D-flat major': '3B',
                    'F minor': '4A', 'A-flat major': '4B', 'C minor': '5A', 'E-flat major': '5B', 'G minor': '6A', 'B-flat major': '6B',
                    'D minor': '7A', 'F major': '7B', 'A minor': '8A', 'C major': '8B', 'E minor': '9A', 'G major': '9B',
                    'B minor': '10A', 'D major': '10B', 'F-sharp minor': '11A', 'A major': '11B', 'D-flat minor': '12A', 'E major': '12B'}

    # Convert diatonic key signatures to Camelot wheel equivalents.

    df['camelot'] = df['key_signature'].map(key_to_wheel)
    df = df.drop(columns=['key', 'mode'])



In [ ]:
# Load Camelot wheel, represented as json data.
with open('camelot.json') as json_file:
    camelot_json = json.load(json_file)
    wheel = camelot_json

In [ ]:
key_to_camelot(merged)

In [ ]:
merged

In [ ]:
# Replace str with track ID corresponding to the song you wish to perform queries on.

input_song_id = '6CUk0IQxiFNZRmCKP8t63o'

# Replace line above with line below for a random song.
# input_song_id = merged.iloc[randint(0, len(merged) - 1)]['id']


In [ ]:

song_selected = merged.loc[merged['id'] == input_song_id]
song_selected

In [ ]:
merged = merged[['track_name', 'artist', 'album', 'tempo',
                 'key_signature', 'camelot', 'time_signature', 'duration_ms', 'id']]


In [ ]:
# Select harmonically compatible key signatures in camelot.json 
friendly_keys = wheel[song_selected['camelot'][song_selected.index[0]]]['all']

# Designate desired tempo range
tempo_range = 10
selected_tempo = song_selected['tempo'][song_selected.index[0]]
acceptable_tempos = list(range(selected_tempo - tempo_range, selected_tempo + tempo_range, 1))

# Show tracks with harmonically compatible key signatures within a given tempo range. Accounts for Spotify's tendency to double or halve numeric tempos.
merged.query('camelot in @friendly_keys & (tempo in @acceptable_tempos | tempo * 2 in @acceptable_tempos | tempo / 2 in @acceptable_tempos)')